# Extracting text from images with OCR Tesseract

## OCR com Tesseract

In [1]:
import os
import re
from unidecode import unidecode
from unicodedata import normalize
from fuzzywuzzy import fuzz
import locale
import time, copy

In [2]:
from pdf2image import convert_from_path

In [89]:
def encontrar_valores3(texto, marcador1, marcador2):
    # Certificando-se de que todos os parâmetros são strings
    if isinstance(texto, list):
        texto = '\n'.join(texto)
    if not isinstance(marcador1, str):
        marcador1 = str(marcador1)
    if not isinstance(marcador2, str):
        marcador2 = str(marcador2)

    # Encontrando o texto entre os dois marcadores
    resultado = re.search(f'{re.escape(marcador1)}(.*?){re.escape(marcador2)}', texto, re.DOTALL)
    if resultado:
        segmento = resultado.group(1)
        
        # Encontrando todos os valores no formato R$ 9,99 e as alíquotas como 3%
        valores = re.findall(r'R\$ [\d\.]*\d+,\d{2}|(?:\d+,\d{1,2}|\d+)%', segmento)
        posicoes = [m.start() for m in re.finditer(r'R\$ [\d\.]*\d+,\d{2}|(?:\d+,\d{1,2}|\d+)%', segmento)]

        # Criando uma lista de tuplas (valor, posição)
        valores_com_posicoes = list(zip(valores, posicoes))
        
        return valores_com_posicoes
    else:
        return None

### Pre-processamento

In [373]:
# XXX Funcao que converte PDF para imagem de tamanho grande (4134, 5846) - torna-a cinza e a salva
def convert_resize_gray(original_file_name, file_path, image_resized_path):

    name_image = conv_filename_no_ext(original_file_name)
    image_resized_name = os.path.join(f'{image_resized_path}/{str(name_image)}.jpg')
    pages = convert_from_path(file_path, 500, poppler_path=poppler_path)
    # 4. Verifica se ha mais que uma pagina
    if len(pages) > 1:
        raise ValueError("Erro, documento com mais de uma página")
    else:
        # 5. Iterar pelas páginas e redimensionar
        resized_pages = []
        for page in pages:
            resized_page = page.resize((4134, 5846))
            resized_pages.append(resized_page)
            
    imagem_gray = resized_pages[0].convert('L')
    imagem_gray.save(image_resized_name, 'JPEG')

    return  imagem_gray, image_resized_name

In [374]:
image_resized_path = "pipeline_extracao_documentos/6_geral_administacao/temp_docs/images/processadas"

In [377]:
file_path = "pipeline_extracao_documentos/0_arquivos_teste_pipeline/pdf_pesquisavel/Mesquita/acfrogblgyewspqaweud3qjkpdqn5kp2dfiynq7d6wjcrymgxkby0xaq7m2xyrrh8asjkxsfk1z9f4bsqat1di5gppkc3ahrhnhavaawbjuamkpiluuxpydd2ovrxzk.pdf"
original_file_name = "acfrogblgyewspqaweud3qjkpdqn5kp2dfiynq7d6wjcrymgxkby0xaq7m2xyrrh8asjkxsfk1z9f4bsqat1di5gppkc3ahrhnhavaawbjuamkpiluuxpydd2ovrxzk.pdf"

In [378]:
imagem_gray, image_resized_name = convert_resize_gray(original_file_name, file_path, image_resized_path)

NameError: name 'conv_filename_no_ext' is not defined

In [3]:
import cv2
from PIL import Image
import numpy as np

In [301]:
from PIL import Image
imagem = Image.open("pipeline_extracao_documentos/6_geral_administacao/temp_docs/images/processadas/nf_11_igreja_evangelica_batista_eterna_vida.jpg")

In [ ]:
imagem_gray = imagem.convert('L')
imagem_gray

In [354]:
imagem_gray.save('imagem_gray.jpg')

In [275]:
from PIL import Image

import pytesseract

# If you don't have tesseract executable in your PATH, include the following:
#pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'
# Example tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract'

# 12. poppler path
poppler_path = "/home/dani-boy/miniconda3/envs/tables-detr/bin"

# 6. Path para gestao de imagens resized
image_resized_path = "pipeline_extracao_documentos/6_geral_administacao/temp_docs/images/processadas"


tessdata_dir = '--dpi 600 --oem 3 --psm 4'

In [ ]:
tessdata_dir = '--tessdata-dir "/home/dani-boy/miniconda3/envs/tables-detr/share/tessdata/" --user-patterns "novo_modelo/modelos/user-patterns2.txt" --dpi 600 --oem 3 --psm 6'

In [ ]:
section = "10. OBSERVACOES"
tipo = "field_box"
father_value = "5_frame_inf_criticas"
marcador_ini = "Observação:"

In [ ]:
texto_valores = linhas

# Exemplo de uso
marcador1 = 'VALOR SERVIÇOS:'  # SAO PEDRO DA ALDEIA
marcador2 = 'LOCAL INCIDÊNCIA'
lista_val = encontrar_valores2(texto_valores, marcador1, marcador2)
lista_val

print(f'\nlista_val: {lista_val}  -->  {len(lista_val)}\n')

In [371]:
def encontrar_texto(texto, marcador_inicial, marcador_final=None):
    if marcador_final:
        # Procura pelo texto entre os dois marcadores
        resultado = re.search(f'{re.escape(marcador_inicial)}(.*?){re.escape(marcador_final)}', texto, re.DOTALL)
        if resultado:
            return resultado.group(1).strip()
    else:
        # Procura pelo texto do marcador inicial até o final
        resultado = re.search(f'{re.escape(marcador_inicial)}(.*)', texto, re.DOTALL)
        if resultado:
            return resultado.group(1).strip()

    # Retorna None se não encontrar nada
    return None

In [310]:
def encontrar_valores2(texto, marcador1, marcador2):
    # Certificando-se de que todos os parâmetros são strings
    if isinstance(texto, list):
        texto = '\n'.join(texto)
    if not isinstance(marcador1, str):
        marcador1 = str(marcador1)
    if not isinstance(marcador2, str):
        marcador2 = str(marcador2)

    # Encontrando o texto entre os dois marcadores
    resultado = re.search(f'{re.escape(marcador1)}(.*?){re.escape(marcador2)}', texto, re.DOTALL)
    if resultado:
        segmento = resultado.group(1)
        
        # Encontrando todos os valores no formato R$ 9,99
        valores = re.findall(r'R\$ \d+,\d{2}', segmento)
        
        # Encontrando as posições dos valores na string original
        posicoes = [m.start() for m in re.finditer(r'R\$ \d+,\d{2}', segmento)]
        
        # Criando uma lista de tuplas (valor, posição)
        valores_com_posicoes = list(zip(valores, posicoes))
        
        return valores_com_posicoes
    else:
        return None

In [ ]:
# uncomment the following line(s) to ensure that all packages are available:
#!python -m pip install matplotlib
#!python -m pip install -U pymupdf

%matplotlib inline
import fitz  # import PyMuPDF

def show_image(pix):
    """Display a pixmap using matplotlib and numpy."""
    import matplotlib.pyplot as plt  # needed for displaying intermediate images
    import numpy as np  # needed for displaying intermediate images
    img = np.ndarray((pix.h, pix.w, 3), dtype=np.uint8, buffer=pix.samples_mv)  # raw image to numpy array
    plt.figure(dpi=150)
    image = plt.imshow(img)  # let matplotlib show the image

In [ ]:
# XXX Funcao ajustada para convertere e resize
def convertResize(doc2convert, document_path, image_resized_path):
    
    
    name_image = conv_filename_no_ext(doc2convert)
    
    image_resized_name = os.path.join(f'{image_resized_path}/{str(name_image)}.jpg')
    #print(f'image_resized_name: {image_resized_name}\n')
    # 3. Conversao para imagem
    pages = convert_from_path(document_path, 500, poppler_path=poppler_path)
    
    # 4. Verifica se ha mais que uma pagina
    if len(pages) > 1:
        raise ValueError("Erro, documento com mais de uma página")
    else:
        # 5. Iterar pelas páginas e redimensionar
        resized_pages = []
        for page in pages:
            resized_page = page.resize((2067, 2923))
            resized_pages.append(resized_page)
            
        resized_pages[0].save(image_resized_name, 'JPEG')
        
        image_2work = resized_pages[0]
        
    return image_2work, image_resized_name

# XXX Usando na criacao da imagem 
def conv_filename_no_ext(title):
    
    # Divida o título em nome e extensão (mas ignore a extensão)
    name = title.rsplit('.', 1)[0] if '.' in title else title

    # Remova acentos e caracteres especiais do nome
    name = normalize('NFKD', name).encode('ASCII', 'ignore').decode('ASCII')
    
    # Substitua espaços e hífens por sublinhados
    filename = name.replace(' ', '_').replace('-', '_')

    # Remova quaisquer outros caracteres não alfanuméricos, exceto sublinhados
    filename = re.sub(r'[^\w_]', '', filename)

    # Converter para minúsculas
    filename = filename.lower()

    return filename 


In [ ]:
image_2work, image_path = convertResize(doc2convert, file_path, image_resized_path)

In [ ]:
#tessdata_dir = '--dpi 600 --oem 1 --psm 6'

image = Image.open("4414_cabecalho_3.png")
imagem_gray = image.convert('L')
imagem_gray



In [362]:
texto = (pytesseract.image_to_string(imagem_gray, lang='por'))

In [364]:
texto

'Número da Nota:\n4414\n\nData e Hora de Emissão:\n26/07/2023 - 17:18\n\nCódigo de Verificação:\nF8A952271\n\n'

In [365]:
linhas = [linha for linha in texto.split('\n') if linha.strip()]

In [366]:
linhas

['Número da Nota:',
 '4414',
 'Data e Hora de Emissão:',
 '26/07/2023 - 17:18',
 'Código de Verificação:',
 'F8A952271']

In [340]:
texto = '\n'.join(linhas)

In [341]:
texto

'VALOR SERVIÇOS: VALOR DESC. INCOND: BASE DE ALÍQUOTA: VALOR ISS: VALOR ISS DESC. COND:\nR$ 252.836,00 DEDUÇÃO: R$6,00 CÁLCULO: 2,01% R$ 5.082,00 RETIDO: R$ 508,00\nR$ 20,00 R$ 252.836,00 R$ 0,00\nVALOR PIS: VALOR COFINS: VALOR IR: VALOR INSS: VALOR CSLL: OUTRAS RETENÇÕES: VALOR LÍQUIDO:\nR$3,00 R$36,00 R$ 0,01 R$ 836,00 R$ 0,00 R$66,00 R$ 252.836,00'

In [352]:
valores = re.findall(r'R\$ *([\d\.]+,\d{1,2})|([\d\.]+,\d{1,2})%', texto)
valores_formatados = [f'R$ {x[0]}' if x[0] else f'{x[1]}%' for x in valores]
valores_formatados

['R$ 252.836,00',
 'R$ 6,00',
 '2,01%',
 'R$ 5.082,00',
 'R$ 508,00',
 'R$ 20,00',
 'R$ 252.836,00',
 'R$ 0,00',
 'R$ 3,00',
 'R$ 36,00',
 'R$ 0,01',
 'R$ 836,00',
 'R$ 0,00',
 'R$ 66,00',
 'R$ 252.836,00']

In [346]:
posicoes = [m.start() for m in re.finditer(r'R\$ *([\d\.]+,\d{1,2})|([\d\.]+,\d{1,2})%', texto)]
valores_formatados = [f'R$ {x[0]}' if x[0] else f'{x[1]}%' for x in valores]

In [345]:
valores

[('252.836,00', ''),
 ('6,00', ''),
 ('', '2,01'),
 ('5.082,00', ''),
 ('508,00', ''),
 ('20,00', ''),
 ('252.836,00', ''),
 ('0,00', ''),
 ('3,00', ''),
 ('36,00', ''),
 ('0,01', ''),
 ('836,00', ''),
 ('0,00', ''),
 ('66,00', ''),
 ('252.836,00', '')]

In [348]:
valores_formatados = [f'R$ {x[0]}' if x[0] else f'{x[1]}%' for x in valores]

In [349]:
valores_formatados

['R$ 252.836,00',
 'R$ 6,00',
 '2,01%',
 'R$ 5.082,00',
 'R$ 508,00',
 'R$ 20,00',
 'R$ 252.836,00',
 'R$ 0,00',
 'R$ 3,00',
 'R$ 36,00',
 'R$ 0,01',
 'R$ 836,00',
 'R$ 0,00',
 'R$ 66,00',
 'R$ 252.836,00']

In [347]:
posicoes

[87, 110, 126, 132, 152, 162, 171, 185, 285, 292, 300, 308, 318, 326, 334]

In [ ]:
valores = re.findall(r'R\$ \d+,\d{2}|(?:\d+,\d{1,2}|\d+)%', segmento)

In [307]:
def encontrar_valores3(texto, marcador1, marcador2):
    # Certificando-se de que todos os parâmetros são strings
    if isinstance(texto, list):
        texto = '\n'.join(texto)
    if not isinstance(marcador1, str):
        marcador1 = str(marcador1)
    if not isinstance(marcador2, str):
        marcador2 = str(marcador2)

    # Encontrando o texto entre os dois marcadores
    resultado = re.search(f'{re.escape(marcador1)}(.*?){re.escape(marcador2)}', texto, re.DOTALL)
    if resultado:
        segmento = resultado.group(1)
        
        # Encontrando todos os valores no formato R$ 9,99 e as alíquotas como 3%
        valores = re.findall(r'R\$ \d+,\d{2}|(?:\d+,\d{1,2}|\d+)%', segmento)
        
        # Encontrando as posições dos valores na string original
        posicoes = [m.start() for m in re.finditer(r'R\$ \d+,\d{2}|(?:\d+,\d{1,2}|\d+)%', segmento)]
        
        # Criando uma lista de tuplas (valor, posição)
        valores_com_posicoes = list(zip(valores, posicoes))
        
        return valores_com_posicoes
    else:
        return None


In [313]:
# 2. XXX FunÇao para pesquisar entre marcadores do texto
def extrair_texto_entre_marcadores(texto, marcador_inicio, marcador_fim):
    try:
        # Encontra os índices dos marcadores de início e fim
        indice_inicio = next(i for i, s in enumerate(texto) if marcador_inicio in s)
        indice_fim = next(i for i, s in enumerate(texto) if marcador_fim in s)

        # Se o marcador de início e fim estão na mesma linha
        if indice_inicio == indice_fim:
            inicio = texto[indice_inicio].find(marcador_inicio) + len(marcador_inicio)
            fim = texto[indice_fim].find(marcador_fim)
            return texto[indice_inicio][inicio:fim].strip()
        else:
            # Extrai e retorna o texto entre os marcadores
            return " ".join(texto[indice_inicio:indice_fim])
    except StopIteration:
        # Retorna None se algum dos marcadores não for encontrado
        return None

In [367]:
texto

'Número da Nota:\n4414\n\nData e Hora de Emissão:\n26/07/2023 - 17:18\n\nCódigo de Verificação:\nF8A952271\n\n'

In [315]:
texto = (pytesseract.image_to_string(imagem_gray, lang='por'))
linhas = [linha for linha in texto.split('\n') if linha.strip()]
linhas

['Número da Nota:',
 'PREFEITURA MUNICIPAL DE MAGE E',
 'SECRETARIA MUNICIPAL DA FAZENDA Julho/2023',
 'NOTA FISCAL DE SERVIÇOS ELETRÔNICA - NFS-e ata e Hora da Emissão:',
 '28/07/2023 13:16:00',
 'Código Verificação:',
 '3CFDF62AA',
 'PRESTADOR DE SERVIÇOS',
 'CPF/CNPJ: Inscrição Municipal: le:',
 '38.947.654/0001-08 1004403 =',
 'Telefone: Inscrição Estadual:',
 'Nome/Razão Social:',
 'JR DE ARAUJO JIMENEZ',
 'Nome de Fantasia:',
 'Endereço:',
 'RUA BRASIL ,116 ,PIABETA - Magé-RJ',
 'E-mail:',
 'RAFAJIM3SOGMAIL.COM',
 'TOMADOR DE SERVIÇOS',
 'CPF/CNPJ: ,',
 '02.329.932/0001-08 | RG:',
 'INSC:MUNICIPAL:',
 'Telefone: Inscrição Estadual:',
 'Nome/Razão Social:',
 'IGREJA EVANGELICA BATISTA ETERNA VIDA',
 'Endereço: ; .',
 'DOIS Nº 05 BAIRRO: MAURIMÁRCIA (VILA INHOMIRIM) CIDADE: MAGE - RJ CEP: 25930630',
 'E-mail:',
 'NÃO INFORMADO',
 'DISCRIMINAÇÃO DOS SERVIÇOS',
 'Prestação de serviço de elaboração de projeto e desdobro de um lote conforme regras da P.M.M',
 'VALOR TOTAL DA NOTA: R$ 1

In [368]:
def encontrar_texto(texto, marcador_inicial, marcador_final=None):
    if marcador_final:
        # Procura pelo texto entre os dois marcadores
        resultado = re.search(f'{re.escape(marcador_inicial)}(.*?){re.escape(marcador_final)}', texto, re.DOTALL)
        if resultado:
            return resultado.group(1).strip()
    else:
        # Procura pelo texto do marcador inicial até o final
        resultado = re.search(f'{re.escape(marcador_inicial)}(.*)', texto, re.DOTALL)
        if resultado:
            return resultado.group(1).strip()

    # Retorna None se não encontrar nada
    return None




In [369]:
# Testando a função
texto = '''DADOS COMPLEMENTARES
OUTRAS INFORMAÇÕES / CRITICAS
EXIGIBILIDADE ISS REGIME TRIBUTAÇÃO SIMPLES NACIONAL ISSQN RETIDO LOCAL. PRESTAÇÃO LOCAL INCIDÊNCIA
Exigivel Microempresa municipal Sim (2,17% ) Não SERVIÇO Magé - RJ
Magé - RJ
Observação:
- Prestador Optante do Simples Nacional (Alíquota: 2,17 %)
Valor Aproximado dos Tributos Federais R$ 134,5 (Alíq 13,45), Tributos Estaduais R$ 0,00 (Alíg 0,00 IBPT) e Municipal de R$ 48,4 (Alíq IBPT 4,84 IBPT)'''



In [370]:
resultado = encontrar_texto(texto, "Observação:")
print(resultado)

- Prestador Optante do Simples Nacional (Alíquota: 2,17 %)
Valor Aproximado dos Tributos Federais R$ 134,5 (Alíq 13,45), Tributos Estaduais R$ 0,00 (Alíg 0,00 IBPT) e Municipal de R$ 48,4 (Alíq IBPT 4,84 IBPT)


In [326]:
texto_valores = linhas
# Exemplo de uso
marcador_inicio = 'DADOS COMPLEMENTARES'  # SAO PEDRO DA ALDEIA
marcador_fim = 'Observação:'
resultado = extrair_texto_entre_marcadores(texto_valores, marcador_inicio, marcador_fim)
resultado

print(f'\nlista_val: {lista_val}  -->  {len(lista_val)}\n')


lista_val: DADOS COMPLEMENTARES OUTRAS INFORMAÇÕES / CRITICAS EXIGIBILIDADE ISS REGIME TRIBUTAÇÃO SIMPLES NACIONAL ISSQN RETIDO LOCAL. PRESTAÇÃO LOCAL INCIDÊNCIA Exigivel Microempresa municipal Sim (2,17% ) Não SERVIÇO Magé - RJ Magé - RJ  -->  227



In [322]:
resultado = resultado.replace(marcador_inicio, "").strip()

In [329]:
resultado

'DADOS COMPLEMENTARES OUTRAS INFORMAÇÕES / CRITICAS EXIGIBILIDADE ISS REGIME TRIBUTAÇÃO SIMPLES NACIONAL ISSQN RETIDO LOCAL. PRESTAÇÃO LOCAL INCIDÊNCIA Exigivel Microempresa municipal Sim (2,17% ) Não SERVIÇO Magé - RJ Magé - RJ'

In [324]:
lista_val

str

In [256]:
linhas = [s.replace(" ", "").strip() for s in linhas]

In [257]:
linhas

['CoOqMO]',
 'VALORSERVIÇOS:VALOR|DESC.INCOND:BASEDEALÍQUOTA:VALORISS:VALORISSDESC.COND:',
 'R$28.114,98DEDUÇÃO:R$0,00CALCULO:5%R$1.405,75RETIDO:R$0,00',
 'R$0,00R$28.114,98R$0,00']

In [ ]:
resultado = extrair_texto_entre_marcadores(texto_ref, marcador_inicio, marcador_fim)
resultado = resultado.replace(marcador_inicio, "").strip()
resultado_splited = resultado.split(':')
resultado_splited = [s.replace(" ", "").strip() for s in resultado_splited]
resultado_splited = [x for x in resultado_splited if x.strip()]
codido_verificacao = resultado_splited[0]

In [248]:
texto_valores = linhas
# Exemplo de uso
marcador1 = 'VALOR SERVIÇOS'  # SAO PEDRO DA ALDEIA
marcador2 = 'PRESTAÇÃO LOCAL INCIDÊNCIA'
lista_val = encontrar_valores3(texto_valores, marcador1, marcador2)
print(f'\nlista_val: {lista_val} \n')


lista_val: None 



In [236]:

print(f'\nlista_val: {lista_val}  -->  {len(lista_val)}\n')


lista_val: [(('280,00', ''), 75), (('0,00', ''), 94), (('8,40', ''), 114), (('0,00', ''), 130), (('0,00', ''), 142), (('280,00', ''), 150), (('0,00', ''), 160), (('0,00', ''), 260), (('0,00', ''), 268), (('0,00', ''), 276), (('0,00', ''), 284), (('0,00', ''), 292), (('0,00', ''), 300), (('280,00', ''), 308)]  -->  14



In [238]:
# Teste a função com seu texto e marcadores
texto = linhas
marcador1 = 'VALOR SERVIÇOS'
marcador2 = 'DADOS COMPLEMENTARES'

print(encontrar_valores(texto, marcador1, marcador2))

None


[('R$ 17.401,88', 75), ('R$ 0,00', 97), ('2,64%', 114), ('R$ 0,00', 120), ('R$ 0,00', 136), ('R$ 0,00', 144), ('R$ 17.401,88', 152), ('R$ 459,41', 165), ('R$ 0,00', 267), ('R$ 0,00', 275), ('R$ 0,00', 283), ('R$ 0,00', 291), ('R$ 0,00', 299), ('R$ 0,00', 307), ('R$ 16.942,47', 315)]


In [196]:
def encontrar_valores(texto, marcador1, marcador2):
    if isinstance(texto, list):
        texto = '\n'.join(texto)
    if not isinstance(marcador1, str):
        marcador1 = str(marcador1)
    if not isinstance(marcador2, str):
        marcador2 = str(marcador2)
    
    resultado = re.search(f'{re.escape(marcador1)}(.*?){re.escape(marcador2)}', texto, re.DOTALL)
    
    if resultado:
        segmento = resultado.group(1)
        
        matches = re.findall(r'R\$ *([\d\.]*\d*,\d{1,2})|([\d\.]*\d*,\d{1,2})%', segmento)
        posicoes = [m.start() for m in re.finditer(r'R\$ *([\d\.]*\d*,\d{1,2})|([\d\.]*\d*,\d{1,2})%', segmento)]
        
        valores_formatados = [('R$ ' + x[0]) if x[0] else (x[1] + '%') for x in matches]
        
        valores_com_posicoes = list(zip(valores_formatados, posicoes))
        
        return valores_com_posicoes
    else:
        return None




In [199]:
def obter_palavras_e_indices(texto):
    if isinstance(texto, list):
        texto = '\n'.join(texto)

    # Substituindo quebras de linha e múltiplos espaços por um único espaço
    texto = re.sub(r'\s+', ' ', texto)

    # Dividindo o texto em palavras
    palavras = texto.split(' ')

    # Obtendo uma lista de tuplas contendo cada palavra e seu índice
    palavras_com_indices = [(idx, palavra) for idx, palavra in enumerate(palavras) if palavra]

    return palavras_com_indices

In [ ]:
# Teste a função com seu texto
texto = '''... (seu texto aqui) ...'''
print(obter_palavras_e_indices(texto))

In [192]:

def encontrar_valores3(texto, marcador1, marcador2):
    # Certificando-se de que todos os parâmetros são strings
    if isinstance(texto, list):
        texto = '\n'.join(texto)
    if not isinstance(marcador1, str):
        marcador1 = str(marcador1)
    if not isinstance(marcador2, str):
        marcador2 = str(marcador2)

    # Encontrando o texto entre os dois marcadores
    resultado = re.search(f'{re.escape(marcador1)}(.*?){re.escape(marcador2)}', texto, re.DOTALL)
    if resultado:
        segmento = resultado.group(1)
        
        
        valores = re.findall(r'R\$ *([\d\.]+,\d{1,2})|([\d\.]+,\d{1,2})%', segmento)
        posicoes = [m.start() for m in re.finditer(r'R\$ *([\d\.]+,\d{1,2})|([\d\.]+,\d{1,2})%', segmento)]

        # Formatar os valores para incluir o símbolo 'R$' onde necessário
        valores_formatados = [f'R$ {x[0]}' if x[0] else f'{x[1]}%' for x in valores]

        # Criando uma lista de tuplas (valor, posição)
        valores_com_posicoes = list(zip(valores_formatados, posicoes))





        # # Encontrando todos os valores no formato R$ 9,99 e as alíquotas como 3%
        # valores = re.findall(r'R\$ [\d\.]*\d+,\d{2}|(?:\d+,\d{1,2}|\d+)%', segmento)
        # posicoes = [m.start() for m in re.finditer(r'R\$ [\d\.]*\d+,\d{2}|(?:\d+,\d{1,2}|\d+)%', segmento)]

        # Criando uma lista de tuplas (valor, posição)
        valores_com_posicoes = list(zip(valores, posicoes))
        
        return valores_com_posicoes
    else:
        return None


In [52]:
text_splited = texto.split('\n')

In [ ]:
text_splited

In [61]:
texto_ref = [x for x in text_splited if x.strip()]

In [112]:
# Nota 5 - Mesquita
file_path = "pipeline_extracao_documentos/2_documentos_para_extracao/21_aguardando_processamento/Batch_20/MAGE_PDF_31282023_2254/1001204/Nota Magno 150.pdf"
doc2convert = "Nota Magno 150.pdf"

image_2work, image_path = convertResize(doc2convert, file_path, image_resized_path)
imagem_gray = image_2work.convert('L')
texto_doc = (pytesseract.image_to_string(imagem_gray, lang='por'))
text_splited = texto_doc.split('\n')
texto_refNota5 = [x for x in text_splited if x.strip()]
texto_refNota5

NameError: name 'convertResize' is not defined

In [ ]:
texto_ref

In [44]:
linhas_ordenadas = sorted(text_splited)

In [63]:
linhas_ordenadas = sorted(texto_ref, key=len)


In [ ]:
linhas_ordenadas

In [ ]:
linhas

In [65]:
from datetime import datetime

def extrair_data(linha):
    datas = re.findall(r'\d{2}/\d{2}/\d{4}', linha)
    if datas:
        return datetime.strptime(datas[0], '%d/%m/%Y')
    else:
        return datetime.min

In [69]:
linhas_ordenadas = sorted(linhas, key=extrair_data)

In [ ]:
linhas_ordenadas

In [71]:
linhas_ordenadas = sorted(linhas, key=extrair_numero)

In [72]:
linhas_ordenadas

['CPF/CNPJ:',
 'Telefone:',
 'Nome/Razão Social:',
 'LOTERIA SURUI LTDA -ME',
 'Nome de Fantasia:',
 'Endereço:',
 'E-mail:',
 'CPF/CNPJ:',
 'INSC:MUNICIPAL:',
 'Telefone:',
 'Nome/Razão Social:',
 'CAIXA ECONÔMICA FEDERAL',
 'Endereço:',
 'E-mail:',
 'retrispOcaixa.gov.br',
 'VALOR SERVIÇOS:',
 'VALOR PIS:',
 'R$ 0,00',
 'EXIGIBILIDADE ISS',
 'Exigivel',
 'Observação:',
 'Nota Fiscal de Serviços Eletrônica (NFSe)',
 'Número da Nota:',
 'PREFEITURA MUNICIPAL DE MAGE ea E',
 'NOTA FISCAL DE SERVIÇOS ELETRÔNICA - NFS-e',
 'SECRETARIA MUNICIPAL DA FAZENDA',
 'PRESTADOR DE SERVIÇOS',
 'Inscrição Municipal:',
 'Inscrição Estadual:',
 'RG:',
 'TOMADOR DE SERVIÇOS',
 'Inscrição Estadual:',
 'DISCRIMINAÇÃO DOS SERVIÇOS',
 'ata e Hora da Emissão:',
 'Código Verificação:',
 'k',
 'lira',
 'VALOR',
 'DEDUÇÃO:',
 'R$ 0,00',
 'VALOR COFINS: VALOR IR:',
 'R$ 0,00',
 'REGIME TRIBUTAÇÃO',
 'Sociedade Limitada',
 'CUPONS DE APOSTAS, SORTEIOS, PREMIOS, INCLUSIVE OS DECORRENTES DE TÍTULOS DE CAPITALIZAÇÃ

In [ ]:
texto_ref = [x for x in text_splited if x.strip()]

In [ ]:
linhas_ordenadas

In [34]:
texto_valores = texto

# Exemplo de uso
marcador1 = 'TOTAL DA NOTA'  # SAO PEDRO DA ALDEIA
marcador2 = 'LOCAL INCIDÊNCIA'
lista_val = encontrar_valores3(texto_valores, marcador1, marcador2)


print(f'\nlista_val: {lista_val}  -->  {len(lista_val)}\n')


lista_val: [('R$ 3.441,64', 2), ('R$ 0,00', 30), ('R$ 0,00', 64), ('R$ 0,00', 248), ('R$ 169,33', 288), ('R$ 0,00', 591), ('3%', 608), ('R$ 3.441,64', 611), ('R$ 0,00', 647), ('R$ 0,00', 655), ('R$ 0,00', 663), ('3%', 746), ('R$ 103,25', 803), ('R$ 0,00', 821), ('R$ 0,00', 829), ('R$ 0,00', 857), ('R$ 3.441,64', 865)]  -->  17



In [ ]:
text_splited = texto.split('\n')
texto_ref = [x for x in text_splited if x.strip()]
texto_ref

In [ ]:
# Portuguese text image to string
texto = (pytesseract.image_to_string(Image.open(image_path), lang='por'))

In [ ]:
texto

In [ ]:
text_splited = texto.split('\n')
texto_ref = [x for x in text_splited if x.strip()]
texto_ref

In [ ]:
text_splited = texto.split('\n')
texto_ref = [x for x in text_splited if x.strip()]
texto_ref

In [50]:
# Codigo de verificaçao
marcador_inicio = 'Código Verificação:'  # SAO PEDRO DA ALDEIA
marcador_fim = 'PRESTADOR DE SERVIÇOS'

resultado = extrair_texto_entre_marcadores(texto_ref, marcador_inicio, marcador_fim)
resultado = resultado.replace(marcador_inicio, "").strip()
resultado_splited = resultado.split(':')
resultado_splited = [s.replace(" ", "").strip() for s in resultado_splited]
resultado_splited = [x for x in resultado_splited if x.strip()]
codido_verificacao = resultado_splited[0]
print(codido_verificacao)

4ADEE6A7B


In [52]:
# Codigo de verificaçao
marcador_inicio = 'Código Verificação'
marcador_fim = 'PRESTADOR DE SERVIÇOS'

resultado = extrair_texto_entre_marcadores(texto_ref, marcador_inicio, marcador_fim)
resultado = resultado.replace(marcador_inicio, "").strip()
resultado_splited = resultado.split(':')
resultado_splited = [s.replace(" ", "").strip() for s in resultado_splited]
resultado_splited = [x for x in resultado_splited if x.strip()]
codido_verificacao = resultado_splited[0]
print(codido_verificacao)

4ADEE6A7B


In [54]:
# 5. Valor total da nota
marcador_inicio = 'VALOR TOTAL DA NOTA:'
marcador_fim = 'CNAE -'

resultado = extrair_texto_entre_marcadores(text_splited, marcador_inicio, marcador_fim)
valor_total = resultado.replace(marcador_inicio, "").strip()
valor_total_numerico = format_number(valor_total)
print(f'valor_total: {valor_total}    valor_total_numerico: {valor_total_numerico}' )

valor_total: R$ 252.836,00    valor_total_numerico: 252836.0


In [55]:
# 5. CNAE
marcador_inicio = 'CNAE - '
marcador_fim = 'Item da Lista de Serviços -'

resultado = extrair_texto_entre_marcadores(text_splited, marcador_inicio, marcador_fim)
valor_cnae = resultado.replace(marcador_inicio, "").strip()
print(valor_cnae)

4930202 - TRANSPORTE RODOVIÁRIO DE CARGA, EXCETO PRODUTOS PERIGOSOS E MUDANÇAS, INTERMUNICIPAL, INTERESTADUAL


In [164]:
# 6. Items da lista de serviços
marcador_inicio = 'Item da Lista de Serviços -'
marcador_fim = 'VALOR SERVIÇOS:'

resultado = extrair_texto_entre_marcadores(lista, marcador_inicio, marcador_fim)
valor_item_servico = resultado.replace(marcador_inicio, "").strip()
print(valor_item_servico)

NameError: name 'extrair_texto_entre_marcadores' is not defined

In [57]:
# Outras Informacoes criticas
marcador_inicio = 'EXIGIBILIDADE ISS REGIME TRIBUTAÇÃO SIMPLES NACIONAL ISSQN RETIDO LOCAL. PRESTAÇÃO LOCAL INCIDÊNCIA'
marcador_fim = 'Observação:'

resultado = extrair_texto_entre_marcadores(text_splited, marcador_inicio, marcador_fim)
resultado
resultado = resultado.replace(marcador_inicio, "").strip()
resultado

'Exigivel Sociedade Limitada Sim (2,01% ) Não SERVIÇO Magé - RJ Magé - RJ'

In [58]:
# valores dos impostos
marcador_inicio = 'VALOR ISS: VALOR ISS DESC. COND:'
marcador_fim = 'DADOS COMPLEMENTARES'
resultado = extrair_texto_entre_marcadores(text_splited, marcador_inicio, marcador_fim)

In [59]:
resultado

'VALOR SERVIÇOS: VALOR | DESC. INCOND: BASE DE ALÍQUOTA: VALOR ISS: VALOR ISS DESC. COND: R$ 252.836,00 DEDUÇÃO: R$ 0,00 CÁLCULO: 2,01% R$ 5.082,00 RETIDO: R$ 0,00  R$ 0,00 R$ 252.836,00 R$ 0,00 VALOR PIS: VALOR COFINS: VALOR IR: VALOR INSS: VALOR CSLL: OUTRAS RETENÇÕES: VALOR LÍQUIDO: R$ 0,00 R$ 0,00 R$ 0,00 R$ 0,00 R$ 0,00 R$ 0,00 R$ 252.836,00 '

In [60]:
# valores dos impostos
marcador_inicio = 'VALOR ISS'
marcador_fim = 'VALOR COFINS:'

resultado = extrair_texto_entre_marcadores(text_splited, marcador_inicio, marcador_fim)
resultado
resultado = resultado.replace(marcador_inicio, "").strip()
resultado

'VALOR SERVIÇOS: VALOR | DESC. INCOND: BASE DE ALÍQUOTA: :  DESC. COND: R$ 252.836,00 DEDUÇÃO: R$ 0,00 CÁLCULO: 2,01% R$ 5.082,00 RETIDO: R$ 0,00  R$ 0,00 R$ 252.836,00 R$ 0,00'

In [ ]:
resultado

In [ ]:
resultado

In [61]:
# Outras Informacoes criticas
marcador_inicio = 'EXIGIBILIDADE ISS REGIME TRIBUTAÇÃO SIMPLES NACIONAL ISSQN RETIDO LOCAL. PRESTAÇÃO LOCAL INCIDÊNCIA'
marcador_fim = 'Observação:'

resultado = extrair_texto_entre_marcadores(text_splited, marcador_inicio, marcador_fim)
s = resultado.replace(marcador_inicio, "").strip()
s

'Exigivel Sociedade Limitada Sim (2,01% ) Não SERVIÇO Magé - RJ Magé - RJ'

In [38]:
def split_string_with_exceptions(s, phrases):
    safe_phrases = {phrase: phrase.replace(' ', '_') for phrase in phrases}

    for phrase, safe_phrase in safe_phrases.items():
        s = s.replace(phrase, safe_phrase)

    words = s.split()

    for safe_phrase, phrase in safe_phrases.items():
        words = [word.replace(safe_phrase, phrase) for word in words]

    return words

def get_original_phrase(safe_word, safe_phrases_to_original):
    return safe_phrases_to_original.get(safe_word, safe_word)

def remove_word_from_list(word_list, word_to_remove):
    return [word for word in word_list if word != word_to_remove]

In [ ]:
s

In [ ]:
# cadastro a lista de termos inseparaveis
phrases = ["Sociedade Limitada", "Magé - RJ"]

In [ ]:
# Processo o split
result = split_string_with_exceptions(s, phrases)
print(result)

In [ ]:
result[]

In [ ]:
safe_phrases_to_original = {"Sociedade_Limitada": "Sociedade Limitada", "Magé_-_RJ": "RJ - Magé"}

In [ ]:
# Obter novamente a palavra da lista em seu formato original
original_phrase = get_original_phrase("Magé_-_RJ", safe_phrases_to_original)
print(original_phrase)  # Deve imprimir: "Sociedade Limitada"

In [ ]:
# Usando a função
word_list = ["Sociedade Limitada", "Magé - RJ"]

In [ ]:
# Teste
s = "Sociedade Limitada é uma forma de constituição de empresas onde o capital social não se encontra diretamente ligado à pessoa dos sócios, permitindo que uma pessoa possa participar de várias empresas. A sede fica em RJ Magé."

In [ ]:
# Usando a função
safe_phrases_to_original = {"Sociedade_Limitada": "Sociedade Limitada", "RJ_Magé": "RJ Magé"}

In [ ]:
word_to_remove = "Sociedade_Limitada"

In [ ]:
new_result = remove_word_from_list(result, word_to_remove)

In [ ]:
print(new_word_list)  # Deve imprimir: ["RJ_Magé", "outra_palavra"]

In [ ]:
extract_number(valor_total)

In [ ]:
format_number(valor_total)

In [37]:
def format_number(number_str):
    # Check for percentage and handle it
    if '%' in number_str:
        number_str = number_str.replace('%', '')
        return float(number_str)  # You can multiply by 100 here if needed

    # Check if the string contains "R$" or a comma, indicating the original format
    if 'R$' in number_str or ',' in number_str:
        # Original format: Remove 'R$', replace dots with nothing, and replace commas with dots
        number_str = number_str.replace('R$', '').replace('.', '').replace(',', '.')
    else:
        # New format: Extract only the numeric part using regex
        number_str = re.findall(r'[\d\.]+', number_str)[-1]

    return float(number_str)

def extract_number(text):
    match = re.search(r'\b\d+(\.\d+)?\b', text)
    if match:
        return match.group(0)
    else:
        return None

In [ ]:
resultado

In [ ]:
resultado_splited = resultado.split(':')

In [ ]:
resultado_splited = resultado.split(':')
resultado_splited = [s.replace(" ", "").strip() for s in resultado_splited]
resultado_splited = [x for x in resultado_splited if x.strip()]
codido_verificacao = resultado_splited[0]
print(codido_verificacao)

In [ ]:
codido_verificacao = resultado_splited[0]
print(codido_verificacao)

In [ ]:
print(codido_verificacao)

In [ ]:
resultado_splited = [x for x in resultado_splited if x.strip()]

In [ ]:
text_splited = texto.split(':')
text_splited = [x for x in text_splited if x.strip()]
text_splited = [s.replace(";", "").strip() for s in text_splited] #depende da situaçao

In [ ]:
resultado_splited = [s.replace(" ", "").strip() for s in resultado_splited] #depende da situaçao

In [ ]:
resultado_splited = [x for x in resultado_splited if x.strip()]

In [ ]:
    text_splited = texto.split('\n')
    text_splited = [x for x in text_splited if x.strip()]
    text_splited = [s.replace(";", "").strip() for s in text_splited] #depende da situaçao
    return text_splited

In [ ]:
text_splited = [x for x in text_splited if x.strip()]

In [ ]:
text_splited = [s.replace(";", "").strip() for s in text_splited] #depende da situaçao

In [ ]:
# Portuguese text image to string
print(pytesseract.image_to_string(Image.open(image_path), lang='por'))

In [ ]:
# Get bounding box estimates
print(pytesseract.image_to_boxes(Image.open(image_path)))

In [ ]:
# Get verbose data including boxes, confidences, line and page numbers
print(pytesseract.image_to_data(Image.open(image_path)))

In [ ]:
# Get information about orientation and script detection
print(pytesseract.image_to_osd(Image.open(image_path)))

## Batch processs

In [ ]:
# Batch processing with a single file containing the list of multiple image file paths
print(pytesseract.image_to_string(image_path))

# Timeout/terminate the tesseract job after a period of time
try:
    print(pytesseract.image_to_string(image_path, timeout=2)) # Timeout after 2 seconds
    print(pytesseract.image_to_string(image_path, timeout=0.5)) # Timeout after half a second
except RuntimeError as timeout_error:
    # Tesseract processing is terminated
    pass

In [ ]:
# Get HOCR output
#hocr = pytesseract.image_to_pdf_or_hocr(image_path, extension='hocr')

# Get ALTO XML output
xml = pytesseract.image_to_alto_xml(image_path)

In [ ]:
xml

In [ ]:
import numpy as np
import cv2

In [ ]:
img = image_path

In [ ]:
norm_img = np.zeros((img.shape[0], img.shape[1]))

In [ ]:
img = cv2.normalize(img, norm_img, 0, 255, cv2.NORM_MINMAX)

In [ ]:
cv2.CALIB_CB_NORMALIZE_IMAGE = np.zeros((img.shape[0], img.shape[1]))

In [ ]:
norm_img = np.zeros((img.shape[0], img.shape[1]))
img = cv2.normalize(img, norm_img, 0, 255, cv2.NORM_MINMAX)
img = cv2.threshold(img, 100, 255, cv2.THRESH_BINARY)[1]
img = cv2.GaussianBlur(img, (1, 1), 0)

In [ ]:
from pytesseract import Output
import pytesseract
import cv2

In [ ]:
filename = image_path
image = cv2.imread(filename)

In [ ]:
results = pytesseract.image_to_data(image, 
output_type=Output.DICT)

In [ ]:
print(results)

In [ ]:
for i in range(0, len(results['text'])):
   x = results['left'][i]
   y = results['top'][i]

   w = results['width'][i]
   h = results['height'][i]

   text = results['text'][i]
   conf = int(results['conf'][i])

   if conf > 70:
        text = ''.join([c if ord(c) < 128 else '' for c in text]).strip()
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(image, text, (x, y - 10), 
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 200), 2)
        cv2.imshow(winname, mat)

In [ ]:
 text

In [ ]:
cv2.imshow(winname, mat)

In [ ]:
cv2.imshow('', frame)

In [ ]:
cv2.imshow(image, mat)

In [ ]:
import numpy as np
import cv2

image = np.full((300, 300, 3), 255).astype(np.uint8)

cv2.putText(image, 'some picture', (20, 60),
            cv2.FONT_HERSHEY_SIMPLEX, 1, [0, 0, 0])

cv2.imshow('custom window name', image)
cv2.waitKey(0)

In [ ]:
import cv2 as cv 

path = R'cabecalho.png'

img = cv.imread(path)

cv.imshow("Dog", img)

cv.waitKey(0)

In [ ]:
"""
Demo script using Tesseract OCR.

Extract text of a page and interpret unrecognized characters using Tesseract.
MuPDF codes unrecognizable characters as 0xFFFD = 65533.
Extraction option is "dict", which delivers contiguous text pieces within one
line, that have the same font properties (color, fontsize, etc.). Together with
the language parameter, this helps Tesseract finding the correct character.

The basic approach is to only invoke OCR, if the span text contains
chr(65533). Because Tesseract's response ignores leading spaces and appends
line break characters, some adjustments are made.

--------------
This demo will OCR only text, that is known to be text. This means, it
does not look at parts of a page containing images or text encoded as drawings.
--------------

Dependencies:
Tesseract must be installed and invocable via Python's 'subprocess' module.
You also must install all the Tesseract language support you need to detect.
"""
import fitz
import subprocess
import time

# Tesseract invocation command (Windows version)
# Assume: language English. Detect more languages by add e.g. '+deu' for German.
# Assume: text represents one line (--psm 7)
# Note: Language mix spec increases duration by >40% - only use when needed!
tess = "tesseract stdin stdout --psm 7 -l eng"
mat = fitz.Matrix(4, 4)  # high resolution matrix
ocr_time = 0
pix_time = 0


def get_tessocr(page, bbox):
    """Return OCR-ed span text using Tesseract.

    Args:
        page: fitz.Page
        bbox: fitz.Rect or its tuple
    Returns:
        The OCR-ed text of the bbox.
    """
    global ocr_time, pix_time, tess, mat
    # Step 1: Make a high-resolution image of the bbox.
    t0 = time.perf_counter()
    pix = page.get_pixmap(
        colorspace=fitz.csGRAY,  # we need no color
        matrix=mat,
        clip=bbox,
    )
    image = pix.tobytes("png")  # make a PNG image
    t1 = time.perf_counter()
    # Step 2: Invoke Tesseract to OCR the image. Text is stored in stdout.
    rc = subprocess.run(
        tess,  # the command
        input=image,  # the pixmap image
        stdout=subprocess.PIPE,  # find the text here
        shell=True,
    )

    # because we told Tesseract to interpret the image as one line, we now need
    # to strip off the line break characters from the tail.
    text = rc.stdout.decode()  # convert to string
    text = text[:-3]  # remove line end characters
    t2 = time.perf_counter()
    ocr_time += t2 - t1
    pix_time += t1 - t0
    return text




In [ ]:
file = r".\data\Hold_em_test\Notas Magé\NF 688 - ConsoR Zadar- Engetecnica (ENZA).pdf"

In [ ]:
doc = fitz.open(file)

In [ ]:
doc

In [ ]:
ocr_count = 0
for page in doc:
    print(page)
    get_tessocr(page, 10)

In [ ]:
blocks

In [ ]:

ocr_count = 0
for page in doc:
    blocks = page.get_text("dict", flags=0)["blocks"]
    for b in blocks:
        for l in b["lines"]:
            for s in l["spans"]:
                text = s["text"]
                if chr(65533) in text:  # invalid characters encountered!
                    # invoke OCR
                    ocr_count += 1
                    print("before: '%s'" % text)
                    text1 = text.lstrip()
                    sb = " " * (len(text) - len(text1))  # leading spaces
                    text1 = text.rstrip()
                    sa = " " * (len(text) - len(text1))  # trailing spaces
                    new_text = sb + get_tessocr(page, s["bbox"]) + sa
                    print(" after: '%s'" % new_text)

print("-------------------------")
print("OCR invocations: %i." % ocr_count)
print(
    "Pixmap time: %g (avg %g) seconds."
    % (round(pix_time, 5), round(pix_time / ocr_count, 5))
)
print(
    "OCR time: %g (avg %g) seconds."
    % (round(ocr_time, 5), round(ocr_time / ocr_count, 5))
)